In [0]:
dbutils.fs.ls('/FileStore/tables/nyc_taxi_csv')

[FileInfo(path='dbfs:/FileStore/tables/nyc_taxi_csv/nyc_taxi.csv', name='nyc_taxi.csv', size=17174468, modificationTime=1701221318000)]

In [0]:
# dbutils.fs.rm('/FileStore/tables/nyc_taxi_csv', True)

True

In [0]:
%sql
describe schema default

database_description_item,database_description_value
Catalog Name,spark_catalog
Namespace Name,default
Comment,Default Hive database
Location,dbfs:/user/hive/warehouse
Owner,public


In [0]:
%sql
describe schema dlt_loan_demo_gbell

database_description_item,database_description_value
Catalog Name,spark_catalog
Namespace Name,dlt_loan_demo_gbell
Comment,
Location,dbfs:/user/hive/warehouse/dlt_loan_demo_gbell.db
Owner,root


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS dbt_glue_db 
    COMMENT 'This is schema/database for DBT-Glue' 
    LOCATION 'dbfs:/user/hive/warehouse/dbt_glue.db'
    WITH DBPROPERTIES (Author='lchawathe', Usage='Dev/Test');

In [0]:
%sql
describe schema dbt_glue_db

database_description_item,database_description_value
Catalog Name,spark_catalog
Namespace Name,dbt_glue_db
Comment,This is schema/database for DBT-Glue
Location,dbfs:/user/hive/warehouse/dbt_glue.db
Owner,root


In [0]:
%sql
select * from dbt_glue_db.nyc_taxi_csv limit 5;

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01T00:35:40Z,2022-01-01T00:53:29Z,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
2,2022-01-01T00:58:06Z,2022-01-01T01:05:45Z,1.0,1.58,1.0,N,239,151,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
1,2022-01-01T00:59:06Z,2022-01-01T01:12:57Z,2.0,3.8,1.0,Y,233,238,2,13.5,3.0,0.5,0.0,0.0,0.3,17.3,2.5,0.0
2,2022-01-01T00:55:13Z,2022-01-01T01:04:56Z,4.0,2.07,1.0,N,90,45,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01T00:24:13Z,2022-01-01T00:38:50Z,1.0,2.73,1.0,N,211,170,2,12.0,0.5,0.5,0.0,0.0,0.3,15.8,2.5,0.0


In [0]:
csv_s3_path = 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/csv/nyc_taxi.csv'

In [0]:
dbutils.fs.cp('dbfs:/FileStore/tables/nyc_taxi_csv/nyc_taxi.csv', csv_s3_path)

True

In [0]:
dbutils.fs.ls(csv_s3_path)

[FileInfo(path='s3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/csv/nyc_taxi.csv', name='nyc_taxi.csv', size=17174468, modificationTime=1701222130000)]

In [0]:
athena_results_s3_path = 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/athena_results/'
dbutils.fs.mkdirs(athena_results_s3_path)

True

In [0]:
dbutils.fs.ls(athena_results_s3_path)

[]

In [0]:
%sql
CREATE EXTERNAL TABLE IF NOT EXISTS
  dbt_glue_db.nyc_taxi_csv_on_s3
  (
        vendorid bigint,
        tpep_pickup_datetime timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count double,
        trip_distance double,
        ratecodeid double,
        store_and_fwd_flag string,
        pulocationid bigint,
        dolocationid bigint,
        payment_type bigint,
        fare_amount double,
        extra double,
        mta_tax double,
        tip_amount double,
        tolls_amount double,
        improvement_surcharge double,
        total_amount double,
        congestion_surcharge double,
        airport_fee double
  )
  ROW FORMAT DELIMITED
  FIELDS TERMINATED BY ','
  STORED AS TEXTFILE
  LOCATION 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/csv/';

In [0]:
%sql
select * from dbt_glue_db.nyc_taxi_csv_on_s3 limit 5;

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01T00:35:40Z,2022-01-01T00:53:29Z,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
2,2022-01-01T00:58:06Z,2022-01-01T01:05:45Z,1.0,1.58,1.0,N,239,151,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
1,2022-01-01T00:59:06Z,2022-01-01T01:12:57Z,2.0,3.8,1.0,Y,233,238,2,13.5,3.0,0.5,0.0,0.0,0.3,17.3,2.5,0.0
2,2022-01-01T00:55:13Z,2022-01-01T01:04:56Z,4.0,2.07,1.0,N,90,45,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01T00:24:13Z,2022-01-01T00:38:50Z,1.0,2.73,1.0,N,211,170,2,12.0,0.5,0.5,0.0,0.0,0.3,15.8,2.5,0.0


In [0]:
%sql
select day(tpep_pickup_datetime) as PICKUP_DAY, count(*) as PICKUP_COUNT from dbt_glue_db.nyc_taxi_csv_on_s3 group by day(tpep_pickup_datetime) order by 1;

PICKUP_DAY,PICKUP_COUNT
1,4230
2,3893
3,4827
4,4973
5,4972
6,5323
7,4776
8,5544
9,4268
10,4911


In [0]:
iceberg_table_base_s3_path = 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/nyc_taxi_iceberg/'
dbutils.fs.rm(iceberg_table_base_s3_path, True)

print(f'Cleaned Iceberg table from {iceberg_table_base_s3_path}')

Cleaned Iceberg table from s3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/nyc_taxi_iceberg/


In [0]:
%sql
/*
-- Delta does not support expression in partitionby
CREATE TABLE IF NOT EXISTS
  dbt_glue_db.nyc_taxi_iceberg 
  (
        vendorid bigint,
        tpep_pickup_datetime timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count double,
        trip_distance double,
        ratecodeid double,
        store_and_fwd_flag string,
        pulocationid bigint,
        dolocationid bigint,
        payment_type bigint,
        fare_amount double,
        extra double,
        mta_tax double,
        tip_amount double,
        tolls_amount double,
        improvement_surcharge double,
        total_amount double,
        congestion_surcharge double,
        airport_fee double
  )
  PARTITIONED BY (day(tpep_pickup_datetime))
  LOCATION 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/nyc_taxi_iceberg/'
  TBLPROPERTIES ( 'table_type' ='ICEBERG'  );
*/

CREATE TABLE IF NOT EXISTS
  dbt_glue_db.nyc_taxi_iceberg 
  (
        vendorid bigint,
        tpep_pickup_datetime timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count double,
        trip_distance double,
        ratecodeid double,
        store_and_fwd_flag string,
        pulocationid bigint,
        dolocationid bigint,
        payment_type bigint,
        fare_amount double,
        extra double,
        mta_tax double,
        tip_amount double,
        tolls_amount double,
        improvement_surcharge double,
        total_amount double,
        congestion_surcharge double,
        airport_fee double
  )
  LOCATION 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/nyc_taxi_iceberg/'
  TBLPROPERTIES ( 'table_type' ='ICEBERG'  );

In [0]:
%sql
INSERT INTO dbt_glue_db.nyc_taxi_iceberg
  SELECT * FROM dbt_glue_db.nyc_taxi_csv_on_s3;

num_affected_rows,num_inserted_rows
164263,164263


In [0]:
iceberg_load_df = spark.sql('select * from dbt_glue_db.nyc_taxi_iceberg')
display(iceberg_load_df.limit(5))

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01T00:35:40Z,2022-01-01T00:53:29Z,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
2,2022-01-01T00:58:06Z,2022-01-01T01:05:45Z,1.0,1.58,1.0,N,239,151,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
1,2022-01-01T00:59:06Z,2022-01-01T01:12:57Z,2.0,3.8,1.0,Y,233,238,2,13.5,3.0,0.5,0.0,0.0,0.3,17.3,2.5,0.0
2,2022-01-01T00:55:13Z,2022-01-01T01:04:56Z,4.0,2.07,1.0,N,90,45,2,9.0,0.5,0.5,0.0,0.0,0.3,12.8,2.5,0.0
2,2022-01-01T00:24:13Z,2022-01-01T00:38:50Z,1.0,2.73,1.0,N,211,170,2,12.0,0.5,0.5,0.0,0.0,0.3,15.8,2.5,0.0


In [0]:
%sql
SELECT * FROM dbt_glue_db.nyc_taxi_iceberg WHERE day(tpep_pickup_datetime) =  5 limit 10;

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-05T00:06:25Z,2022-01-05T00:18:16Z,1.0,3.6,1.0,N,143,42,2,12.5,3.0,0.5,0.0,0.0,0.3,16.3,2.5,0.0
2,2022-01-05T00:53:53Z,2022-01-05T00:59:37Z,1.0,1.05,1.0,N,79,4,2,6.0,0.5,0.5,0.0,0.0,0.3,9.8,2.5,0.0
2,2022-01-05T00:12:44Z,2022-01-05T00:16:55Z,1.0,0.98,1.0,N,107,170,1,5.5,0.5,0.5,1.4,0.0,0.3,10.7,2.5,0.0
2,2022-01-05T00:08:04Z,2022-01-05T00:30:10Z,1.0,5.8,1.0,N,79,188,1,20.5,0.5,0.5,4.86,0.0,0.3,29.16,2.5,0.0
2,2022-01-05T00:04:33Z,2022-01-05T00:36:25Z,1.0,16.5,1.0,N,132,188,4,-46.0,-0.5,-0.5,0.0,0.0,-0.3,-48.55,0.0,-1.25
1,2022-01-05T00:47:35Z,2022-01-05T00:51:22Z,1.0,0.7,1.0,N,164,137,1,5.0,3.0,0.5,1.75,0.0,0.3,10.55,2.5,0.0
2,2022-01-05T00:23:26Z,2022-01-05T00:27:22Z,1.0,1.13,1.0,N,107,162,1,5.5,0.5,0.5,0.46,0.0,0.3,9.76,2.5,0.0
2,2022-01-05T00:09:48Z,2022-01-05T00:30:27Z,1.0,9.44,1.0,N,132,76,2,27.5,0.5,0.5,0.0,0.0,0.3,30.05,0.0,1.25
2,2022-01-05T00:18:00Z,2022-01-05T00:23:35Z,1.0,0.89,1.0,N,234,68,2,5.5,0.5,0.5,0.0,0.0,0.3,9.3,2.5,0.0
2,2022-01-05T00:20:20Z,2022-01-05T00:31:40Z,1.0,3.58,1.0,N,158,239,1,12.0,0.5,0.5,3.16,0.0,0.3,18.96,2.5,0.0


In [0]:
%sql
-- add pickup_day column to represent day(tpep_pickup_datetime)
CREATE TABLE IF NOT EXISTS
  dbt_glue_db.nyc_taxi_iceberg_data_manipulation 
  (
        vendorid bigint,
        pickup_day INT,
        tpep_pickup_datetime timestamp,
        tpep_dropoff_datetime timestamp,
        passenger_count double,
        trip_distance double,
        ratecodeid double,
        store_and_fwd_flag string,
        pulocationid bigint,
        dolocationid bigint,
        payment_type bigint,
        fare_amount double,
        extra double,
        mta_tax double,
        tip_amount double,
        tolls_amount double,
        improvement_surcharge double,
        total_amount double,
        congestion_surcharge double,
        airport_fee double
  )
  PARTITIONED BY (pickup_day)
  LOCATION 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/nyc_taxi_iceberg_data_manipulation/'
  TBLPROPERTIES ( 'table_type' ='ICEBERG'  );

In [0]:
%sql
INSERT INTO dbt_glue_db.nyc_taxi_iceberg_data_manipulation
    SELECT
        vendorid,
        day(tpep_pickup_datetime) as pickup_day,
        tpep_pickup_datetime,
        tpep_dropoff_datetime,
        passenger_count,
        trip_distance,
        ratecodeid,
        store_and_fwd_flag,
        pulocationid,
        dolocationid,
        payment_type,
        fare_amount,
        extra,
        mta_tax,
        tip_amount,
        tolls_amount,
        improvement_surcharge,
        total_amount,
        congestion_surcharge,
        airport_fee
    FROM dbt_glue_db.nyc_taxi_iceberg;

num_affected_rows,num_inserted_rows
164263,164263


In [0]:
%sql
-- check query from table that supports partitioning
SELECT * FROM dbt_glue_db.nyc_taxi_iceberg_data_manipulation WHERE pickup_day =  5 limit 10;

vendorid,pickup_day,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,5,2022-01-05T00:06:25Z,2022-01-05T00:18:16Z,1.0,3.6,1.0,N,143,42,2,12.5,3.0,0.5,0.0,0.0,0.3,16.3,2.5,0.0
2,5,2022-01-05T00:53:53Z,2022-01-05T00:59:37Z,1.0,1.05,1.0,N,79,4,2,6.0,0.5,0.5,0.0,0.0,0.3,9.8,2.5,0.0
2,5,2022-01-05T00:12:44Z,2022-01-05T00:16:55Z,1.0,0.98,1.0,N,107,170,1,5.5,0.5,0.5,1.4,0.0,0.3,10.7,2.5,0.0
2,5,2022-01-05T00:08:04Z,2022-01-05T00:30:10Z,1.0,5.8,1.0,N,79,188,1,20.5,0.5,0.5,4.86,0.0,0.3,29.16,2.5,0.0
2,5,2022-01-05T00:04:33Z,2022-01-05T00:36:25Z,1.0,16.5,1.0,N,132,188,4,-46.0,-0.5,-0.5,0.0,0.0,-0.3,-48.55,0.0,-1.25
1,5,2022-01-05T00:47:35Z,2022-01-05T00:51:22Z,1.0,0.7,1.0,N,164,137,1,5.0,3.0,0.5,1.75,0.0,0.3,10.55,2.5,0.0
2,5,2022-01-05T00:23:26Z,2022-01-05T00:27:22Z,1.0,1.13,1.0,N,107,162,1,5.5,0.5,0.5,0.46,0.0,0.3,9.76,2.5,0.0
2,5,2022-01-05T00:09:48Z,2022-01-05T00:30:27Z,1.0,9.44,1.0,N,132,76,2,27.5,0.5,0.5,0.0,0.0,0.3,30.05,0.0,1.25
2,5,2022-01-05T00:18:00Z,2022-01-05T00:23:35Z,1.0,0.89,1.0,N,234,68,2,5.5,0.5,0.5,0.0,0.0,0.3,9.3,2.5,0.0
2,5,2022-01-05T00:20:20Z,2022-01-05T00:31:40Z,1.0,3.58,1.0,N,158,239,1,12.0,0.5,0.5,3.16,0.0,0.3,18.96,2.5,0.0


In [0]:
%sql
-- 161 records with passenger_count: Null or 0
-- 2991 records with passenger_count = 1
SELECT passenger_count, count(*) as trips 
FROM dbt_glue_db.nyc_taxi_iceberg_data_manipulation 
WHERE (vendorid = 2 and year(tpep_pickup_datetime) = 2022 and month(tpep_pickup_datetime) = 1 and pickup_day = 31)
GROUP BY passenger_count
ORDER BY passenger_count ASC;

passenger_count,trips
null,160
0.0,1
1.0,2991
2.0,493
3.0,130
4.0,49
5.0,91
6.0,70


In [0]:
%sql
-- Update the passenger_count to fill NULL or 0
UPDATE dbt_glue_db.nyc_taxi_iceberg_data_manipulation 
SET passenger_count = 1
WHERE (vendorid = 2 and year(tpep_pickup_datetime) = 2022 and month(tpep_pickup_datetime) = 1 and pickup_day = 31) AND (passenger_count is NULL OR passenger_count = 0)

num_affected_rows
161


In [0]:
%sql
-- confirm the updates
-- 3152 (2991 + 160 + 1) records with passenger_count = 1
SELECT passenger_count, count(*) as trips 
FROM dbt_glue_db.nyc_taxi_iceberg_data_manipulation 
WHERE (vendorid = 2 and year(tpep_pickup_datetime) = 2022 and month(tpep_pickup_datetime) = 1 and pickup_day = 31)
GROUP BY passenger_count
ORDER BY passenger_count ASC;

passenger_count,trips
1.0,3152
2.0,493
3.0,130
4.0,49
5.0,91
6.0,70


In [0]:
%sql
-- Earliest time is 2023-11-29 02:41:54
SELECT * FROM dbt_glue_db.nyc_taxi_iceberg_data_manipulation FOR SYSTEM_TIME AS OF TIMESTAMP '2023-11-29 03:00:00' WHERE vendorid = 2 and year(tpep_pickup_datetime)= 2022 limit 10; 

vendorid,pickup_day,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee


In [0]:
%sql
describe FORMATTED dbt_glue_db.nyc_taxi_iceberg_data_manipulation;

col_name,data_type,comment
vendorid,bigint,null
pickup_day,int,null
tpep_pickup_datetime,timestamp,null
tpep_dropoff_datetime,timestamp,null
passenger_count,double,null
trip_distance,double,null
ratecodeid,double,null
store_and_fwd_flag,string,null
pulocationid,bigint,null
dolocationid,bigint,null


In [0]:
%sql
/* 
-- Error: UC_NOT_ENABLED
select
    h.made_current_at,
    s.operation,
    h.snapshot_id,
    h.is_current_ancestor,
    s.summary['spark.app.id']
from dbt_glue_db.nyc_taxi_iceberg_data_manipulation.history h
join dbt_glue_db.nyc_taxi_iceberg_data_manipulation.snapshots s
  on h.snapshot_id = s.snapshot_id
order by made_current_at
*/

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: [UC_NOT_ENABLED] Unity Catalog is not enabled on this cluster.
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndIdentifier$.unapply(LookupCatalog.scala:152)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.$anonfun$resolveRelation$1(Analyzer.scala:1671)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveRelations$$resolveRelation(Analyzer.scala:1670)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply0$1.$anonfun$applyOrElse$82(Analyzer.scala:1435)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveRelations$$handleGlueError(Analyzer.scala:1547)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply0$1.applyOrElse(Analyzer.scala:1435)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply0$1.applyOrElse(Analyzer.scala:1391)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$3(AnalysisHelper.scala:141)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$1(AnalysisHelper.scala:141)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:372)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning(AnalysisHelper.scala:137)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning$(AnalysisHelper.scala:133)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUpWithPruning(LogicalPlan.scala:39)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$2(AnalysisHelper.scala:138)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren(TreeNode.scala:1274)
	at org.apache.spark.sql.catalyst.trees.UnaryLike.mapChildren$(TreeNode.scala:1273)
	at org.apache.spark.sql.catalyst.plans.logical.SubqueryAlias.mapChildren(basicLogicalOperators.scala:1924)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$1(AnalysisHelper.scala:138)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:372)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning(AnalysisHelper.scala:137)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning$(AnalysisHelper.scala:133)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUpWithPruning(LogicalPlan.scala:39)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$2(AnalysisHelper.scala:138)
	at org.apache.spark.sql.catalyst.trees.BinaryLike.mapChildren(TreeNode.scala:1301)
	at org.apache.spark.sql.catalyst.trees.BinaryLike.mapChildren$(TreeNode.scala:1300)
	at org.apache.spark.sql.catalyst.plans.logical.Join.mapChildren(basicLogicalOperators.scala:580)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsUpWithPruning$1(AnalysisHelper.scala:138)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:372)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning(AnalysisHelper.scala:137)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.resolveOperatorsUpWithPruning$(AnalysisHelper.scala:133)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUpWithPruning(LogicalPlan.scala:39

In [0]:
modified_iceberg_table_s3_path = 's3://rvo-prevention-coaching-app-stage-us-east-1/testing/dbt-glue-iceberg/nyc_taxi_iceberg_data_manipulation/'

#
# Looks like the table is NOT Iceberg, but just Delta
#

# iceberg_df = spark.read.format("iceberg").load(modified_iceberg_table_s3_path)
iceberg_df = spark.read.format("delta").load(modified_iceberg_table_s3_path)

iceberg_df.printSchema()



root
 |-- vendorid: long (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pulocationid: long (nullable = true)
 |-- dolocationid: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

